In [38]:
#!/usr/bin/env python
import numpy as np
import open3d as o3d

voxel_size = 0.01
max_correspondence_dist_coarse = 15
max_correspondence_dist_fine = 1.5

def load_pc(voxel_size = 0.0):
    pcds = []
    for i in range(1,146):
        if(i % 20 ==0):
            pcd = o3d.io.read_point_cloud("D:/Master tuhh 2020/Sem2/RNM files/www/www/PCD_ datapoints/PCD_ datapoints/%d.pcd" %i)
            pcd.estimate_normals()
            pcd_ds = pcd.voxel_down_sample(voxel_size = voxel_size)
        #print(np.asarray(pcd.points))
            pcds.append(pcd_ds)
    mesh_call = mesh_generation()
    pcds.append(mesh_call)
    return pcds

def pairwise_registration(source, target):
    print("Apply point-to-plane ICP")
    icp_coarse = o3d.pipelines.registration.registration_icp(
        source, target, max_correspondence_distance_coarse, np.identity(4),
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    icp_fine = o3d.pipelines.registration.registration_icp(
        source, target, max_correspondence_distance_fine,
        icp_coarse.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    transformation_icp = icp_fine.transformation
    information_icp = o3d.pipelines.registration.get_information_matrix_from_point_clouds(
        source, target, max_correspondence_distance_fine,
        icp_fine.transformation)
    return transformation_icp, information_icp


def full_registration(pcds, max_correspondence_distance_coarse,
                      max_correspondence_distance_fine):
    pose_graph = o3d.pipelines.registration.PoseGraph()
    odometry = np.identity(4)
    pose_graph.nodes.append(o3d.pipelines.registration.PoseGraphNode(odometry))
    n_pcds = len(pcds)
    for source_id in range(n_pcds):
        for target_id in range(source_id + 1, n_pcds):
            transformation_icp, information_icp = pairwise_registration(
                pcds[source_id], pcds[target_id])
            print("Build o3d.pipelines.registration.PoseGraph")
            if target_id == source_id + 1:  # odometry case
                odometry = np.dot(transformation_icp, odometry)
                pose_graph.nodes.append(
                    o3d.pipelines.registration.PoseGraphNode(
                        np.linalg.inv(odometry)))
                pose_graph.edges.append(
                    o3d.pipelines.registration.PoseGraphEdge(source_id,
                                                             target_id,
                                                             transformation_icp,
                                                             information_icp,
                                                             uncertain=False))
            else:  # loop closure case
                pose_graph.edges.append(
                    o3d.pipelines.registration.PoseGraphEdge(source_id,
                                                             target_id,
                                                             transformation_icp,
                                                             information_icp,
                                                             uncertain=True))
    return pose_graph


def mesh_generation():
    mesh = o3d.io.read_triangle_mesh("D:/Master tuhh 2020/Sem2/RNM files/www/www/scanning/Skeleton_Target.stl")
    point_cld = mesh.sample_points_poisson_disk(100000)

    #o3d.visualisation.draw_geometries([mesh])
    
    #o3d.visualization.draw_geometries([point_cld])
    return point_cld


if __name__ == "__main__":
    pcds_down = load_pc(voxel_size)
    print(len(pcds_down))
    o3d.visualization.draw_geometries(pcds_down)
    
#     print("Full registration ...")
#     max_correspondence_distance_coarse = voxel_size * 15
#     max_correspondence_distance_fine = voxel_size * 1.5
#     with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
#         pose_graph = full_registration(pcds_down,max_correspondence_distance_coarse,
#                                        max_correspondence_distance_fine)

#     print("Optimizing PoseGraph ...")
#     option = o3d.pipelines.registration.GlobalOptimizationOption(
#     max_correspondence_distance=max_correspondence_distance_fine,
#     edge_prune_threshold=0.25,
#     reference_node=0)
#     with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
#         o3d.pipelines.registration.global_optimization(pose_graph,
#             o3d.pipelines.registration.GlobalOptimizationLevenbergMarquardt(),
#             o3d.pipelines.registration.GlobalOptimizationConvergenceCriteria(),
#             option)
        
#     print("Transform points and display")
#     for point_id in range(len(pcds_down)):
#         print(pose_graph.nodes[point_id].pose)
#         pcds_down[point_id].transform(pose_graph.nodes[point_id].pose)
#     o3d.visualization.draw_geometries(pcds_down)


#     pcds = load_pc(voxel_size)
#     pcd_combined = o3d.geometry.PointCloud()
#     for point_id in range(len(pcds)):
#         pcds[point_id].transform(pose_graph.nodes[point_id].pose)
#         pcd_combined += pcds[point_id]
#     pcd_cb = pcd_combined.voxel_down_sample(voxel_size=voxel_size)
#     #o3d.io.write_point_cloud("multiway_registration.pcd", pcd_cb)
#     o3d.visualization.draw_geometries([pcd_cb])

#     skt_pc = mesh_generation()
#     skt_pc.estimate_normals()
#     pcd_cb.estimate_normals()
#     reg_comb = []
#     reg_comb.append(pcd_cb)
#     reg_comb.append(skt_pc)
#     print("Register PC with the skeleton file")
#     max_correspondence_distance_coarse = voxel_size * 15
#     max_correspondence_distance_fine = voxel_size * 1.5
#     with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
#         pose_graph1 = full_registration(reg_comb,max_correspondence_distance_coarse,
#                                        max_correspondence_distance_fine)
        
#     print("Optimizing PoseGraph for the registered file ...")
#     option = o3d.pipelines.registration.GlobalOptimizationOption(
#     max_correspondence_distance=max_correspondence_distance_fine,
#     edge_prune_threshold=0.25,
#     reference_node=0)
#     with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
#         o3d.pipelines.registration.global_optimization(pose_graph,
#             o3d.pipelines.registration.GlobalOptimizationLevenbergMarquardt(),
#             o3d.pipelines.registration.GlobalOptimizationConvergenceCriteria(),
#             option)

#     skt_pc = mesh_generation()
#     skt_pc.estimate_normals()
#     pcd_cb.estimate_normals()
#     pcds_ak= []
#     pcds_ak.append(pcd_cb)
#     pcds_ak.append(skt_pc)
    
#     pcd_final = o3d.geometry.PointCloud()
#     for point_id in range(len(pcds_ak)):
#         pcds_ak[point_id].transform(pose_graph1.nodes[point_id].pose)
#         pcd_final += pcds_ak[point_id]
#     final = pcd_final.voxel_down_sample(voxel_size=voxel_size)
#     #o3d.io.write_point_cloud("multiway_registration.pcd", pcd_cb)
#     print("the final registration")
#     o3d.visualization.draw_geometries([final])
    
    
    
    
    
    
    

8
